In [1]:
using Pkg
Pkg.instantiate()
using QAlgebra

using BenchmarkTools

Precompiling project...
   1233.5 ms  ✓ QAlgebra
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM("eta"), i=(3, QubitPauli("sigma")), b=Ladder())

StateSpace: [α, β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ]
   - SubSpace ["h"]: PM Qubit (Fermionic):  ηᵖₚ, ηᵐₚ, ηᶻₚ, ηᴵₚ
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  σˣₚ, σʸₚ, σᶻₚ, σᴵₚ
   - SubSpace ["b"]: Ladder (Bosonic):  p†p
   - Op: A
   - Op: B(H,U)


In [3]:
var_dict, op_dict, abstract_dict = base_operators(qspace)
alpha = base_operators(qspace, "alpha")
beta = base_operators(qspace, "beta")
gamma_i, gamma_j, gamma_k = base_operators(qspace, "gamma", do_dict=false)
ph, mh, zh = base_operators(qspace, "h", do_dict=false)
xi,yi,zi, pi, mi = base_operators(qspace, "i", do_dict=false)
xj, yj, zj, pj, mj = base_operators(qspace, "j", do_dict=false)
xk, yk, zk, pk, mk = base_operators(qspace, "k", do_dict=false)
b, n = base_operators(qspace, "b", do_dict=false)
I = base_operators(qspace, "I")
A = base_operators(qspace, "A")
println("Done")

Done


In [4]:
A = base_operators(qspace, "A", do_fun=true)
A1 = base_operators(qspace, "A_1", do_fun=false)
A = A()

A

In [5]:
expr = neq(Sum("j", alpha * yi * b + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk)))

β(t)γᵢ²(σˣᵢ+σˣᵢ)+ασʸᵢb+∑ⱼβ(t)γⱼ²σˣᵢ+∑ⱼβ(t)γᵢγⱼσˣⱼ+∑ⱼ⁼ασʸᵢb+∑ₖβ(t)γᵢγₖσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ

In [6]:
expr = simplify(neq(log(Sum("j", alpha * yi * b + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk)))))

log(ασʸᵢb+2β(t)γᵢ²σˣᵢ+∑ⱼ(β(t)γⱼ²σˣᵢ+β(t)γᵢγⱼσˣⱼ+ασʸᵢb)+∑ₖβ(t)γᵢγₖσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [7]:
xj*Sum("k", (beta + beta^2) * alpha^2 * xi * xi * xk)

σˣⱼ∑ₖ⁼(α²β(t)σˣₖ+α²β(t)²σˣₖ)

In [8]:
Sum("j", alpha * (beta+beta^2) * yi * yj + Sum("k", beta * alpha^2 * xi * xi * xk))

∑ⱼ⁼(αβ(t)σʸᵢσʸⱼ+αβ(t)²σʸᵢσʸⱼ+∑ₖ⁼α²β(t)σˣₖ)

In [11]:
@benchmark $alpha * $yi * $yj

BenchmarkTools.Trial: 10000 samples with 7 evaluations per sample.
 Range (min … max):  4.291 μs …  2.962 ms  ┊ GC (min … max):  0.00% … 99.52%
 Time  (median):     4.818 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.622 μs ± 44.377 μs  ┊ GC (mean ± σ):  13.51% ±  1.72%

          ▁▅█▇▆▇▅▅▅▇▆▇▆█▆▅▃▂                                  
  ▁▁▁▁▂▃▄▇██████████████████▇▇▆▆▄▃▃▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▁▂▁▁▁▁▁▁ ▄
  4.29 μs        Histogram: frequency by time        5.96 μs <

 Memory estimate: 11.73 KiB, allocs estimate: 166.

In [9]:
@benchmark Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xj * xk))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  19.306 μs …  21.474 ms  ┊ GC (min … max):  0.00% … 99.56%
 Time  (median):     22.141 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   26.103 μs ± 264.642 μs  ┊ GC (mean ± σ):  14.09% ±  1.41%

         ▁▂▄▅▄▆▅▅▆▆█▇▆▆▅▂▂                                      
  ▁▁▁▂▃▅▇██████████████████▇▅▄▃▃▃▃▂▂▂▂▁▁▂▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▄
  19.3 μs         Histogram: frequency by time         29.3 μs <

 Memory estimate: 54.94 KiB, allocs estimate: 762.

In [10]:
expr = simplify(neq(log(Sum("j", alpha * yi * yj + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk)))))

log(α+2β(t)γᵢ²σˣᵢ+∑ⱼ(β(t)γⱼ²σˣᵢ+β(t)γᵢγⱼσˣⱼ+ασʸᵢσʸⱼ)+∑ₖβ(t)γᵢγₖσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [11]:
simplify(d_dt(xi, expr))

d(σˣᵢ) / dt = log(α+4β(t)γᵢ²σˣᵢ+∑ⱼ(ασʸᵢσʸⱼ+2β(t)γⱼ²σˣᵢ+2β(t)γᵢγⱼσˣⱼ)+∑ₖ2β(t)γᵢγₖσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [6]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))
dA_dt = d_dt(A, expr)
dx_dt = simplify(substitute(A, xi, dA_dt))

d(σˣᵢ) / dt = log(-iασᶻᵢ+4β(t)γᵢ²σˣᵢ+∑ⱼ(2β(t)γⱼ²σˣᵢ+2β(t)γᵢγⱼσˣⱼ+ασˣᵢσʸⱼ)+∑ₖ2β(t)γᵢγₖσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [13]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))

log(ασʸᵢA+2β(t)γᵢ²σˣᵢAσˣᵢ+∑ⱼ(β(t)γⱼ²A+β(t)γᵢγⱼσˣⱼAσˣᵢ+ασʸⱼA)+∑ₖβ(t)γᵢγₖσˣᵢAσˣₖ+∑ⱼₖ⁼β(t)γⱼγₖσˣⱼAσˣₖ)

In [14]:
@benchmark substitute($A, $xi, $dA_dt)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  70.642 μs …  21.016 ms  ┊ GC (min … max):  0.00% … 98.62%
 Time  (median):     78.106 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   93.208 μs ± 382.874 μs  ┊ GC (mean ± σ):  12.95% ±  3.77%

     █▇▄▂▁                                                      
  ▂▃██████▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▁▂▂▂▂▂▂▂▂▁▂▂ ▃
  70.6 μs         Histogram: frequency by time          155 μs <

 Memory estimate: 171.44 KiB, allocs estimate: 2280.

In [15]:
@benchmark d_dt($xi, neq(log(Sum("j", $alpha * $xi  * $yj + Sum("k", $beta * $gamma_j * $gamma_k * $xi * $xj * $xk)))))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  120.185 μs …  21.295 ms  ┊ GC (min … max):  0.00% … 98.64%
 Time  (median):     130.093 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   148.453 μs ± 369.590 μs  ┊ GC (mean ± σ):  11.04% ±  5.54%

       ▁▄▆█▆▅▄▃▂▂▁▁                                              
  ▂▂▃▄▆█████████████▇▇▆▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂ ▄
  120 μs           Histogram: frequency by time          171 μs <

 Memory estimate: 246.72 KiB, allocs estimate: 3109.

In [17]:
println("Before: ", contains_abstract(expr)) 
println("After:  ", contains_abstract(substitute(A, xi, dA_dt)))

Before: true
After:  false


In [18]:
are_indexes_defined(dx_dt)

true

In [7]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))
dA_dt = d_dt(A, expr)
dx_dt = substitute(A, xi, dA_dt)
reorder(dx_dt)

d(σˣᵢ) / dt = log(4β(t)γᵢ²σˣᵢ-iασᶻᵢ+∑ⱼ(2β(t)γⱼ²σˣᵢ+ασˣᵢσʸⱼ+4β(t)γᵢγⱼσˣⱼ)+∑ⱼₖ⁼β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [8]:
@benchmark reorder($dx_dt)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):   8.325 μs …  18.117 ms  ┊ GC (min … max):  0.00% … 99.58%
 Time  (median):     10.640 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   13.242 μs ± 181.194 μs  ┊ GC (mean ± σ):  13.62% ±  1.00%

       ▂▄▅▆██▆▄▄▁                                               
  ▂▄▆████████████▇▆▅▃▃▂▂▂▂▂▁▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  8.32 μs         Histogram: frequency by time         21.7 μs <

 Memory estimate: 35.58 KiB, allocs estimate: 530.

In [19]:
import Base: show, string 
function signed_int_string(i::Int)::String
    if i < -1
        return " -" * string(-i)* " "
    elseif i == -1
        return " - "  
    elseif i == 0
        return " +0" 
    elseif i == 1
        return " + " 
    else
        return " +" * string(i)* " "
    end
end
function intvec_to_braket(is::Vector{Int})::String
    return string("<", join(is, ","), ">")
end

# Define necessary Structs
struct IndexedProduct
    coeff::Int 
    indices::Vector{Vector{Int}}
end

function string(ind_prod::IndexedProduct)::String
    return signed_int_string(ind_prod.coeff)*join(intvec_to_braket.(ind_prod.indices),"")
end
function show(io::IO, ind_prod::IndexedProduct)
    print(io, "IndexedProduct: ", string(ind_prod))
end

struct IndexedCumulant
    n::Int 
    cumulant::Vector{IndexedProduct}
end 
function string(ind_cum::IndexedCumulant)::String
    return join(string.(ind_cum.cumulant), "")
end
function show(io::IO, ind_cum::IndexedCumulant)
    print(io, "IndexedCumulant: ", string(ind_cum))
end

import Base:-
function -(a::IndexedProduct)::IndexedProduct 
    return IndexedProduct(-a.coeff, a.indices)
end

# Define Operations to Construct and Raise the Level of the Structs!
function FirstIndexedCumulant(i::Int=1)
    return IndexedCumulant(1, [IndexedProduct(1, [[i]])])
end
function raising_operator(new_op::Int, prev_product::IndexedProduct)::Vector{IndexedProduct}
    # Raising operator is distributve with respect to products
    # new_op | [i1, i2, i3] = [new_op, i1, i2, i3] - [i1, i2, i3] [new_op]
    # new_op | [[A],[B],...] = [new_op | [A], new_op | [B], ...]
    # The sum of applying itto each Vector{Int} in the product
    n = length(prev_product.indices)
    prev_coeff = prev_product.coeff

    new_product::Vector{IndexedProduct} = []
    for i in 1:length(prev_product.indices)
        new_c = [ copy(v) for v in prev_product.indices ]
        push!(new_c[i], new_op)
        push!(new_product, IndexedProduct(prev_coeff, new_c))
    end
    c = [ copy(v) for v in prev_product.indices ]
    push!(c, [new_op])
    push!(new_product, IndexedProduct(-n*prev_coeff, c))
    return new_product 
end

function raising_operator(new_op::Int, prev_cumulant::IndexedCumulant)
    # Raising operator is distributve with respect to products  
    # and linear with respect to sum   
    cumulant = prev_cumulant.cumulant
    new_cumulant = raising_operator(new_op, cumulant[1]) 
    for i in 2:length(cumulant)
        append!(new_cumulant, raising_operator(new_op, cumulant[i]))
    end
    return IndexedCumulant(prev_cumulant.n+1, new_cumulant)
end
function IndexedCumulant(order::Int)
    if order == 0 
        error("Order must be greater than 0")
    end
    cum = FirstIndexedCumulant()
    for i in 2:order
        cum = raising_operator(i, cum)
    end
    return cum
end
# Test 
cum = IndexedCumulant(3)

IndexedCumulant:  + <1,2,3> - <1,2><3> - <1,3><2> - <1><2,3> +2 <1><2><3>

In [20]:
struct ReducedIndexedCumulant
    operator::IndexedProduct
    approximation::Vector{IndexedProduct}
end
function ReducedIndexedCumulant(order::Int)
    full_cumulant = IndexedCumulant(order)
    operator = full_cumulant.cumulant[1]
    approximation = .-full_cumulant.cumulant[2:end]
    return ReducedIndexedCumulant(operator, approximation)
end
function string(cumulant_list::ReducedIndexedCumulant)
    return string(cumulant_list.operator) * " ≈ " * join(string.(cumulant_list.approximation), "")
end
function show(io::IO, cumulant_list::ReducedIndexedCumulant)
    print(io, "ReducedIndexedCumulant: ", string(cumulant_list))
end
# Test 
ReducedIndexedCumulant(3)

ReducedIndexedCumulant:  + <1,2,3> ≈  + <1,2><3> + <1,3><2> + <1><2,3> -2 <1><2><3>

In [21]:
# Create Cumulant list 
mutable struct ReducedCumulantList
    curr_order::Int
    max_order::Int
    last_cumulant::IndexedCumulant
    reduced_cumulants::Vector{ReducedIndexedCumulant}
end
function string(cumulant_list::ReducedCumulantList)
    return join(string.(cumulant_list.reduced_cumulants), "\n")
end
function show(io::IO, cumulant_list::ReducedCumulantList)
    print(io, "ReducedCumulantList with curr_order = ", cumulant_list.curr_order, "\n", string(cumulant_list))

end


function ReducedCumulantList(curr_order::Int=1;max_order::Int=10^12)
    if curr_order > max_order
        error("Current order must be less than or equal to max order")
    end
    if curr_order > 0 
        reduced_cumulants::Vector{ReducedIndexedCumulant} = []
        full_cumulant = FirstIndexedCumulant()
        operator = full_cumulant.cumulant[1]
        approximation = .-full_cumulant.cumulant[2:end]
        push!(reduced_cumulants, ReducedIndexedCumulant(operator, approximation))
        for i in 2:curr_order
            full_cumulant = raising_operator(i, full_cumulant)
            operator = full_cumulant.cumulant[1]
            approximation = .-full_cumulant.cumulant[2:end]
            push!(reduced_cumulants, ReducedIndexedCumulant(operator, approximation))
        end
        return ReducedCumulantList(curr_order, max_order, full_cumulant, reduced_cumulants)
    else
        error("Current order must be greater than 0")
    end
end
# Access as a function i.e. reduced_cumulant_list(3) gives the 3rd order cumulant and if necessary constructs it up to that order
function (cumulant_list::ReducedCumulantList)(order::Int)
    if order > cumulant_list.curr_order
        expand_cumulant_list!(cumulant_list, order)
    end
    return cumulant_list.reduced_cumulants[order]
end

function expand_cumulant_list!(cumulant_list::ReducedCumulantList, order::Int)::Nothing
    if order > cumulant_list.max_order
        error("Order must be less than or equal to max_order")
    end
    if order <= cumulant_list.curr_order
        warning("Order can only be increased if target order is greater than current order")
        return
    end
    full_cumulant = cumulant_list.last_cumulant
    for i in cumulant_list.curr_order+1:order
        full_cumulant = raising_operator(i, full_cumulant)
        operator = full_cumulant.cumulant[1]
        approximation = .-full_cumulant.cumulant[2:end]
        push!(cumulant_list.reduced_cumulants, ReducedIndexedCumulant(operator, approximation))
    end
    cumulant_list.curr_order = order
    cumulant_list.last_cumulant = full_cumulant
    return
end

# Test 
red_cum = ReducedCumulantList(2)
red_cum(3)
red_cum

ReducedCumulantList with curr_order = 3
 + <1> ≈ 
 + <1,2> ≈  + <1><2>
 + <1,2,3> ≈  + <1,2><3> + <1,3><2> + <1><2,3> -2 <1><2><3>

In [22]:
function where_acting_index(q::QTerm, statespace::StateSpace)::Vector{Int}
    return [i for (i, op) in enumerate(q.op_indices) if op!=statespace.neutral_op[i]]
end
# Order of QTerm 
function order(q::QTerm, statespace::StateSpace)::Int
    # Determines the order of a QTerm operator 
    return length(where_acting_index(q, statespace))
end
# Test 
q = (xi*b*yj).terms[1].expr[1]
println(where_acting_index(q, qspace))
println(order(q, qspace))

[2, 3, 5]
3


In [ ]:
struct QCumulant <:QComposite  # Must be QComposite to be in QExpr's
    statespace::StateSpace
    atom::QAtom
    expr_::QExpr
    order::Int
    where_acting::Vector{Int}
end

In [ ]:
Is = Vector{Int}
function replace_indexes(neutral_opVector{Vector{Int}}, curr_op_indexesVector{Vector{Int}}, indexes::Vector{Int}) 
    new_op = copy(neutral_op)
    for ind in indexes
        new_op[ind] = copy(curr_op_indexes[ind])
    end
    return new_op
end

# Add conspiracies later!
function Cumulant(qprod::QAtomProduct)
    if length(qprod.terms) != 1
        error("Cumulant only defined for single QAtom in QAtomProduct")
    end
    return Cumulant(qprod.coeff_fun, qprod.terms[1].expr[1], qprod.statespace)
end
function Cumulant(q::QAbstract, statespace::StateSpace, kwargs...)::QCumulant
    error("Cumulant not defined for qAbstract!")
end
function Cumulant(coeff_fun::CFunction, atom::QTerm, statespace::StateSpace)::QCumulant
    where_acting = where_acting_index(atom, statespace)
    order = length(where_acting)
    red_cum = ReducedIndexedCumulant(order)
    return _Cumulant(coeff_fun, atom, statespace, order, where_acting, red_cum)
end

function Cumulant(qprod::QAtomProduct, red_cum_list::ReducedCumulantList)::QCumulant
    if length(qprod.terms) != 1
        error("Cumulant only defined for single QAtom in QAtomProduct")
    end
    return Cumulant(qprod.coeff_fun, qprod.terms[1].expr[1], qprod.statespace, red_cum_list)
end
function Cumulant(coeff_fun::CFunction, atom::QTerm, statespace::StateSpace, red_cum_list::ReducedCumulantList)
    where_acting = where_acting_index(atom, statespace)
    order = length(where_acting)
    return _Cumulant(coeff_fun, atom, statespace, order, where_acting, red_cum_list(order))
end

@inline function _Cumulant(coeff_fun::CFunction, atom::QTerm, statespace::StateSpace, order::Int, where_acting::Vector{Int}, red_cum::ReducedIndexedCumulant)::QCumulant
    curr_op_indexes = atom.op_indices
    neutral_op = statespace.neutral_op
    # create indexed cumulant to this order 
    # create QExpr from this 
    qexpr::Vector{qComposite} = Vector{qComposite}(undef, length(red_cum.approximation))
    for (j, app) in enumerate(red_cum.approximation)
        # create qTerm from app, where acting and atom 
        coeff = app.coeff
        indexes = app.indices
        curr_atoms = Vector{QTerm}(undef, length(indexes))
        # replace indexes from neutral with tthose in qterm 
        for (i, ind) in enumerate(indexes)
            curr_atoms[i] = QTerm(neutral_op, replace_indexes(neutral_op, curr_op_indexes, ind))
        end
        qexpr[j] = QAtomProduct(curr_atoms, coeff_fun*coeff, statespace)
    end
    return QCumulant(statespace, copy(atom), qexpr, order, where_acting)
end



_Cumulant (generic function with 1 method)

In [ ]:
# Test Cumulant here 


In [ ]:
# Add cumulants                        [x] 
# add distribution sampling            

In [ ]:
# Add Expand 
# Add Magnus Expansion 
# Ad BCH Expansion 
# Add Taylor Expansion

In [10]:
# Add:
# - Eq_Set 
# - Indexed Eq_Set
#    - Indexed qAtom <: qComposite (not really but better to be gruped as such to not require any qAtomProducts)
#    - evaluate 
#    - Initial States (from function)
#        - From state construction via operators. 
#        -> Via operators allows us to also use operator conspiracies => Matching/Conspiring operator condition.
# Operator Transformations 
#    - Finding Adjoint Cycles 
#    - Add Rotating Frame Transformation 

In [ ]:
# CFunction add exponential and sin and cos and log  -> simplifications can be added later. such as log(exp(...))


In [ ]:
# add a way to lower sort keys for operators by comparing with the adjoint operators sort_key. 


In [9]:
simplify(QCommutator(Sum("i", alpha*ph*xi*yi) + zj,zh))

[zⱼ+∑ᵢ⁼iαpₕzᵢ, zₕ]

In [10]:
simplify(exp(Sum("i", alpha*ph*xi*yi) + zj)+zh)

zₕ+exp(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [11]:
log(Sum("i", alpha*ph*xi*yi) + zj)

log(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [12]:
simplify(power(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)²

In [13]:
simplify(root(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)⁼²